In [97]:
import pandas as pd
import numpy as np
import collections
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
import csv
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split

# Python script for confusion matrix creation.
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.metrics import f1_score

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

In [98]:
data = pd.read_csv('data2.csv')
data.head()

,No,ID,JK,Kecamatan,Jenis_Tinggal,Alat_Transportasi,Penerima_KPS,Pekerjaan_Ayah,Penghasilan_Ayah,Pekerjaan_Ibu,Penghasilan_Ibu,Rombel_Saat_Ini,Penerima_KIP,Layak_PIP
0,1,1,L,Kec. Tenayan Raya,Bersama orang tua,Sepeda motor,Tidak,PNS/TNI/Polri,2-5,PNS/TNI/Polri,2-5,X MIPA 6,Tidak,Tidak
1,2,2,L,Kec. Tenayan Raya,Bersama orang tua,Sepeda motor,Tidak,Wiraswasta,2-5,Pedagang Kecil,"<0,5",XI MIPA 7,Tidak,Tidak
2,3,3,L,Kec. Tenayan Raya,Bersama orang tua,Ojek,Tidak,Sudah Meninggal,no,Tidak bekerja,no,XI IPS 3,Tidak,Tidak
3,4,4,L,Kec. Marpoyan Damai,Bersama orang tua,Sepeda motor,Tidak,Karyawan Swasta,5-20,Tidak bekerja,no,XII MIPA 3,Tidak,Tidak
4,5,5,P,Kec. Tenayan Raya,Bersama orang tua,Angkutan umum/bus/pete-pete,Tidak,Wiraswasta,"0,5-1",Tidak bekerja,no,X IPS 1,Tidak,Ya


In [99]:
data.shape

(882, 14)

In [100]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882 entries, 0 to 881
Data columns (total 14 columns):
No                   882 non-null int64
ID                   882 non-null int64
JK                   882 non-null object
Kecamatan            882 non-null object
Jenis_Tinggal        882 non-null object
Alat_Transportasi    882 non-null object
Penerima_KPS         882 non-null object
Pekerjaan_Ayah       882 non-null object
Penghasilan_Ayah     882 non-null object
Pekerjaan_Ibu        882 non-null object
Penghasilan_Ibu      882 non-null object
Rombel_Saat_Ini      882 non-null object
Penerima_KIP         882 non-null object
Layak_PIP            882 non-null object
dtypes: int64(2), object(12)
memory usage: 55.2+ KB


In [101]:
data = data.drop(['ID'], axis=1)
data = data.drop(['No'], axis=1)
data = data.drop(['JK'], axis=1)
data = data.drop(['Kecamatan'], axis=1)
data = data.drop(['Rombel_Saat_Ini'], axis=1)

In [102]:
data['Jenis_Tinggal'] = data['Jenis_Tinggal'].replace({'Bersama orang tua': 1, 'Wali': 2, 'Asrama': 3,'Lainnya':4,'Kost':4})

data['Alat_Transportasi'] = data['Alat_Transportasi'].replace({'Sepeda motor': 1, 'Ojek': 2, 'Angkutan umum/bus/pete-pete': 3,'Jalan kaki':4,'Mobil/bus antar jemput':5,'Kendaraan pribadi':6,'Lainnya':7,'Mobil pribadi':8,'Kereta api':9})

data['Penerima_KPS'] = data['Penerima_KPS'].replace({'Ya': 1, 'Tidak': 0})

data['Pekerjaan_Ayah'] = data['Pekerjaan_Ayah'].replace({'Wiraswasta': 1, 'PNS/TNI/Polri': 2, 'Sudah Meninggal': 3, 'Karyawan Swasta': 4, 'Pensiunan': 5, 'Pedagang Kecil': 6, 'Petani': 7, 'Tidak bekerja': 8, 'Buruh': 9, 'Buruh': 10, 'Lainnya': 11, 'Wirausaha': 12,'Pedagang Besar':13,'Peternak':14})

data['Penghasilan_Ayah'] = data['Penghasilan_Ayah'].replace({'2-5': 1, 'no': 2, '5-20': 3, '0,5-1': 4, '1-2': 5, '>2': 6, '>2': 7, '<1': 8, '<0,5': 7,'>20':8})

data['Pekerjaan_Ibu'] = data['Pekerjaan_Ibu'].replace({'Wiraswasta': 1, 'PNS/TNI/Polri': 2, 'Sudah Meninggal': 3, 'Karyawan Swasta': 4, 'Pensiunan': 5, 'Pedagang Kecil': 6, 'Petani': 7, 'Tidak bekerja': 8, 'Buruh': 9, 'Buruh': 10, 'Lainnya': 11, 'Wirausaha': 12,'Pedagang Besar':13,'Peternak':14})

data['Penghasilan_Ibu'] = data['Penghasilan_Ibu'].replace({'2-5': 1, 'no': 2, '5-20': 3, '0,5-1': 4, '1-2': 5, '>2': 6, '>2': 7, '<1': 8, '<0,5': 7,'>20':8})

data['Penerima_KIP'] = data['Penerima_KIP'].replace({'Ya': 1, 'Tidak': 0})

data['Layak_PIP'] = data['Layak_PIP'].replace({'Ya': 1, 'Tidak': 0})

features = data.drop(['Layak_PIP'], axis=1)
label = data['Layak_PIP']

In [103]:
data.head()

,Jenis_Tinggal,Alat_Transportasi,Penerima_KPS,Pekerjaan_Ayah,Penghasilan_Ayah,Pekerjaan_Ibu,Penghasilan_Ibu,Penerima_KIP,Layak_PIP
0,1,1,0,2,1,2,1,0,0
1,1,1,0,1,1,6,7,0,0
2,1,2,0,3,2,8,2,0,0
3,1,1,0,4,3,8,2,0,0
4,1,3,0,1,4,8,2,0,1


In [104]:
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2)

In [105]:
gnb = GaussianNB()
nbtrain = gnb.fit(X_train, y_train)

from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier = classifier.fit(X_train,y_train)

In [106]:
pred = nbtrain.predict(X_test)

In [107]:
print("Accuracy Naive Bayes:",metrics.accuracy_score(y_test, pred))
print(classification_report(y_test,pred))

Accuracy Naive Bayes: 0.8926553672316384
              precision    recall  f1-score   support

           0       0.93      0.96      0.94       159
           1       0.46      0.33      0.39        18

    accuracy                           0.89       177
   macro avg       0.69      0.64      0.66       177
weighted avg       0.88      0.89      0.88       177



In [108]:
predd = classifier.predict(X_test)

In [109]:
print('Accuracy Score C4.5:', metrics.accuracy_score(y_test,predd))
print(classification_report(y_test,predd))

Accuracy Score C4.5: 0.8870056497175142
              precision    recall  f1-score   support

           0       0.92      0.96      0.94       159
           1       0.42      0.28      0.33        18

    accuracy                           0.89       177
   macro avg       0.67      0.62      0.64       177
weighted avg       0.87      0.89      0.88       177



In [110]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500)
mlp.fit(X_train,y_train)

c:\users\lizah\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(8, 8, 8), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [111]:
preddd = mlp.predict(X_test)

In [112]:
print('Accuracy Score MLP:', metrics.accuracy_score(y_test,preddd))
print(classification_report(y_test,preddd))

Accuracy Score MLP: 0.9152542372881356
              precision    recall  f1-score   support

           0       0.92      0.99      0.95       159
           1       0.71      0.28      0.40        18

    accuracy                           0.92       177
   macro avg       0.82      0.63      0.68       177
weighted avg       0.90      0.92      0.90       177

